In [15]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import re

def extract_category_keywords(business_numbers):
    
    driver = webdriver.Chrome()

    category_keywords_dict = {}

    for business_number in business_numbers:
        business_number_clean = business_number.replace("-", "")
        
        address = 'https://bizno.net/article/' + business_number_clean
        print(f"접속 중인 URL: {address}")

        driver.get(address)

        # 상호명 추출
        try:
            shop_name = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/div[1]/div[1]/div/div[1]/div/a/h1').text
        except NoSuchElementException:
            shop_name = "상호명 없음"  
        
        # category_keywords 추출
        category_keywords = None
        try:
            element = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/div[1]/div[1]/div/table/tbody/tr[2]/td')
            if all(label in element.text for label in ["대분류", "중분류", "소분류", "세분류", "세세분류"]):
                category_keywords = element.text
        except NoSuchElementException:
            pass
        
        if not category_keywords:
            try:
                element = driver.find_element(By.XPATH, '/html/body/section[2]/div/div/div[1]/div[1]/div/table/tbody/tr[4]/td')
                if all(label in element.text for label in ["대분류", "중분류", "소분류", "세분류", "세세분류"]):
                    category_keywords = element.text
            except NoSuchElementException:
                pass
        
        if category_keywords:
            print(f"업태: {category_keywords}")
        
            category_dict = {
                "상호명": shop_name,
                "대분류": re.search(r"대분류\s*:\s*(.*?)(?=\s*중분류|$)", category_keywords),
                "중분류": re.search(r"중분류\s*:\s*(.*?)(?=\s*소분류|$)", category_keywords),
                "소분류": re.search(r"소분류\s*:\s*(.*?)(?=\s*세분류|$)", category_keywords),
                "세분류": re.search(r"세분류\s*:\s*(.*?)(?=\s*세세분류|$)", category_keywords),
                "세세분류": re.search(r"세세분류\s*:\s*(.*)", category_keywords)
            }

            for key in category_dict:
                if isinstance(category_dict[key], re.Match):
                    category_dict[key] = category_dict[key].group(1).strip()
                
                elif category_dict[key] is not None:
                    category_dict[key] = category_dict[key].strip()
            
            category_keywords_dict[business_number] = category_dict
        else:
            print(f"사업자 번호 {business_number}에 대한 정보를 찾을 수 없습니다.")

    driver.quit()

    return category_keywords_dict

def save_to_json(data, file_path):
    """
    데이터를 JSON 파일로 저장하는 함수.
    
    Args:
        data (dict): 저장할 데이터.
        file_path (str): 저장할 JSON 파일 경로.
    """
    with open(file_path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)
    print(f"데이터가 {file_path}에 JSON 형식으로 저장되었습니다.")

category_keywords = extract_category_keywords(business_numbers)

if category_keywords:
    json_file_path = 'category_keywords.json'
    save_to_json(category_keywords, json_file_path)
else:
    print("카테고리 키워드가 추출되지 않아 JSON 파일로 저장되지 않았습니다.")


접속 중인 URL: https://bizno.net/article/2198101636
업태: 대분류 : 운수 및 창고업
중분류 : 육상 운송 및 파이프라인 운송업
소분류 : 육상 여객 운송업
세분류 : 시외버스 운송업
세세분류 : 시외버스 운송업
데이터가 category_keywords.json에 JSON 형식으로 저장되었습니다.
